In [29]:
!hostname

jupyterhub-3bce7452adc3c0-jupyterhub-0


In [30]:
import os
import torch
import pandas as pd
import argparse
from pytube import YouTube

In [31]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cpu device


In [32]:
kwargs = {'num_workers': 1, 'pin_memory': True} if device=='cuda' else {}
kwargs

{}

In [33]:
DIR_DATASETS = '/home/jupyter-admin/datasets'
DIR_DATASET_MSASL = '/MS-ASL'
DIR_VIDEOS = '/videos'

In [34]:
files = os.listdir(DIR_DATASETS)
files

['.ipynb_checkpoints', 'MS-ASL']

'**url**': a url link to the video<br/>
'**start_time**': the starting point of the clip in the original video in seconds<br/>
'**end_time**': the starting point of the clip in the original video in seconds<br/>
'**label**': class (an integer between 0 to 1000)<br/>
'**signer_id**': the id of the signer<br/>
'**box**': the boudy bounding box of the signer such as [y0, x0, y1, x1] where (x0, y0) is up-left corner and (x1,y1) is bottom-right corner. All the values are normalized (between zero and one) according to width and height.<br/>
'**text**': the gloss for this clip which match the 'label',<br/>
'**width**': height for the original video<br/>
'**height**': height for the original video<br/>
'**fps**': frame per second for the original video<br/>

In [35]:
classes_json = pd.read_json(DIR_DATASETS + DIR_DATASET_MSASL + '/MSASL_classes.json')
classes_json.head()

,0
0,hello
1,nice
2,teacher
3,eat
4,no


In [36]:
synonym_json = pd.read_json(DIR_DATASETS + DIR_DATASET_MSASL + '/MSASL_synonym.json')
synonym_json.head()

,0,1,2,3,4
0,father,dad,daddy,None,None
1,mother,mom,mommy,None,None
2,shoes,shoe,None,None,None
3,pants,pant,None,None,None
4,color,colors,None,None,None


In [37]:
train_json = pd.read_json(DIR_DATASETS + DIR_DATASET_MSASL + '/MSASL_train.json')
train_json.head()

,org_text,clean_text,start_time,signer_id,signer,start,end,file,label,height,fps,end_time,url,text,box,width,review
0,match [light-a-MATCH],match,0.0,0,0,0,83,match light-a-MATCH,830,360,30.00,2.767,https://www.youtube.com/watch?v=C37R_Ix8-qs,match,"[0.057544618844985004, 0.21637457609176602, 1....",640,NaN
1,FAIL,fail,0.0,0,-1,0,74,FAIL,542,360,25.00,2.960,https://www.youtube.com/watch?v=PIsUJl8BN_I,fail,"[0.06577941775321901, 0.167171776294708, 0.939...",480,NaN
2,laugh,laugh,0.0,4,26,0,31,SignSchool Laugh with Legs 2,312,360,29.97,1.034,www.youtube.com/watch?v=9FdHlMOnVjg,laugh,"[0.131885945796966, 0.32334136962890603, 1.0, ...",640,NaN
3,BOOK,book,0.0,0,-1,0,66,BOOK(3),38,360,25.00,2.640,https://www.youtube.com/watch?v=J7tP98oDxqE,book,"[0.055698871612548, 0.25173279643058705, 0.996...",480,NaN
4,sign-language,sign language,0.0,0,-1,0,75,SIGN-LANGUAGE-S-CLAW-F,848,360,29.97,2.502,www.youtube.com/watch?v=N2mG9ZKjrGA,sign language,"[0.039043992757797005, 0.24198183417320202, 1....",640,NaN


In [38]:
test_json = pd.read_json(DIR_DATASETS + DIR_DATASET_MSASL + '/MSASL_test.json')
test_json.head()

,org_text,clean_text,start_time,signer_id,signer,start,end,file,label,height,fps,end_time,url,text,box,width,review
0,beer,beer,0.0,20,40,0,59,SignSchool Beer var,805,360,29.970,1.969,www.youtube.com/watch?v=wX78EPtSuzU,beer,"[0.046787232160568, 0.290409207344055, 1.0, 0....",640,NaN
1,enjoy,enjoy,0.0,11,39,0,54,enjoy(1),192,360,29.970,1.802,www.youtube.com/watch?v=OL02Odh2dRg,enjoy,"[0.071450918912887, 0.136024981737136, 1.0, 0....",480,NaN
2,emotional,emotional,0.0,20,40,0,30,SignSchool Emotional,907,360,23.976,1.251,www.youtube.com/watch?v=C59jcSo4fEI,emotional,"[0.059554219245910006, 0.281019657850265, 1.0,...",640,NaN
3,key,key,0.0,92,-1,0,37,SignSchool Key,456,360,23.976,1.543,www.youtube.com/watch?v=Qs2ua1S6tg0,key,"[0.11462894082069301, 0.308987438678741, 1.0, ...",640,NaN
4,bad,bad,0.0,11,39,0,36,BAD,81,360,29.970,1.201,www.youtube.com/watch?v=-kgTBeOw95A,bad,"[0.040453493595123007, 0.22047379612922602, 0....",480,NaN


In [39]:
val_json = pd.read_json(DIR_DATASETS + DIR_DATASET_MSASL + '/MSASL_val.json')
val_json.head()

,org_text,clean_text,start_time,signer_id,signer,start,end,file,label,height,fps,end_time,url,text,box,width,review
0,absent,absent,0.0,114,-1,0,37,ASL ABSENT,837,360,28.971,1.277,https://www.youtube.com/watch?v=ri3NrdgfAtE,absent,"[0.21896389126777602, 0.008568197488784, 0.997...",202,NaN
1,help,help,0.0,76,42,0,110,help 2,50,360,29.970,3.670,www.youtube.com/watch?v=l31UXgChCS4,help,"[0.050372719764709, 0.29941257834434504, 1.0, ...",640,NaN
2,come on,come on,0.0,114,-1,0,41,asl come on,889,360,25.000,1.640,https://www.youtube.com/watch?v=pt9bV_EvcaU,come on,"[0.08946925401687601, 0.17948511242866502, 0.9...",480,NaN
3,LANGUAGE,language,0.0,3,-1,0,56,LANGUAGE(3),513,360,15.000,3.733,https://www.youtube.com/watch?v=-j1wozf6o9w,language,"[0.177085787057876, 0.003668457269668, 1.0, 0....",480,NaN
4,confused,confused,0.0,53,-1,0,95,ASL Confused,272,360,29.969,3.170,https://www.youtube.com/watch?v=y8tHmOQcCwU,confused,"[0.06262531876564001, 0.209987848997116, 1.0, ...",640,NaN


In [40]:
def download_video(url):
    try:
        video = YouTube(url, use_oauth=True, allow_oauth_cache=True)
        video = video.streams.get_highest_resolution()
        video.download(DIR_DATASETS + DIR_DATASET_MSASL + DIR_VIDEOS)
    except:
        print('Failed to download video' + ' ' + url)


In [41]:
def download_data(x):
    url = x['url']
    name = url.split('v=')[1]
    if not os.path.isfile(DIR_DATASETS + DIR_DATASET_MSASL + DIR_VIDEOS + '/' + name):
        download_video(url)

In [42]:
train_json.apply(lambda x: download_data(x), axis = 1)

Failed to download video www.youtube.com/watch?v=9FdHlMOnVjg
Failed to download video https://www.youtube.com/watch?v=1AyT77LqJzQ
Failed to download video https://www.youtube.com/watch?v=1AyT77LqJzQ
Failed to download video https://www.youtube.com/watch?v=cJOyCgIKyeA
Failed to download video www.youtube.com/watch?v=zqfcp6NRKpA
Failed to download video www.youtube.com/watch?v=7y5Ye-2-ZBs
Failed to download video www.youtube.com/watch?v=rYW7VFPi9_w
Failed to download video https://www.youtube.com/watch?v=0Beq_NIDj2c
Failed to download video https://www.youtube.com/watch?v=0Beq_NIDj2c
Failed to download video www.youtube.com/watch?v=AoQAPgEUIAs
Failed to download video www.youtube.com/watch?v=25BrmaKbzYo
Failed to download video https://www.youtube.com/watch?v=CZkLkFnsNBA
Failed to download video https://www.youtube.com/watch?v=U_nbv5Mq00c
Failed to download video www.youtube.com/watch?v=trqyl20lEno
Failed to download video www.youtube.com/watch?v=oySXUMX7nRY
Failed to download video www.

0        None
1        None
2        None
3        None
4        None
         ... 
16049    None
16050    None
16051    None
16052    None
16053    None
Length: 16054, dtype: object